In [1]:
import sys
from pathlib import Path

# Go up one level to reach OptiMindTune
project_root = Path.cwd().resolve().parents[0]
print("Project root:", project_root)

# Add to sys.path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))


Project root: /home/meher/Desktop/Personal Projects/OptiMindTune_1/OptiMindTune


In [2]:
"""
OptiMind: AI-Powered Model Optimization System
Main entry point for the optimization benchmark.
"""

import logging
import time
from datetime import datetime
from pathlib import Path

from core.orchestrator import OptimizationOrchestrator
from core.data_manager import DatasetManager
from core.results_manager import ResultsManager
from config import OptiMindConfig
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)


def main():
    """Main execution function."""
    try:
        load_dotenv(override=True)
        # Initialize configuration
        config = OptiMindConfig()
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Initialize managers
        data_manager = DatasetManager()
        results_manager = ResultsManager(config.logs_dir, config.csv_dir, timestamp)
        orchestrator = OptimizationOrchestrator(config, results_manager)
        
        logger.info("Starting OptiMind optimization benchmark")
        
        # Load datasets
        datasets = data_manager.load_datasets("iris")
        logger.info(f"Loaded {len(datasets)} datasets: {list(datasets.keys())}")
        
        # Run optimization for each dataset
        benchmark_results = {}
        
        for dataset_name, (X, y) in datasets.items():
            logger.info(f"\n{'='*50}")
            logger.info(f"Starting optimization for {dataset_name} dataset")
            logger.info(f"{'='*50}")
            
            start_time = time.time()
            
            try:
                result = orchestrator.optimize_dataset(dataset_name, X, y, timestamp)
                optimization_time = time.time() - start_time
                
                # Store results
                benchmark_results[dataset_name] = {
                    **result,
                    "optimization_time": optimization_time,
                    "iterations_per_second": result["total_iterations"] / optimization_time
                }
                
                logger.info(f"Completed {dataset_name} optimization in {optimization_time:.2f}s")
                
            except Exception as e:
                logger.error(f"Failed to optimize {dataset_name}: {e}")
                benchmark_results[dataset_name] = {
                    "error": str(e),
                    "optimization_time": time.time() - start_time
                }
        
        # Save final results
        results_manager.save_benchmark_summary(benchmark_results)
        results_manager.print_summary(benchmark_results)
        
        logger.info("OptiMind benchmark completed successfully")
        
    except Exception as e:
        logger.error(f"Critical error in main execution: {e}")
        raise

main()

INFO:__main__:Starting OptiMind optimization benchmark
INFO:core.data_manager:Loaded iris dataset: 150 samples, 4 features
INFO:__main__:Loaded 1 datasets: ['iris']
INFO:__main__:
INFO:__main__:Starting optimization for iris dataset
INFO:__main__:==================================================
INFO:core.orchestrator:Initializing optimization for iris
INFO:core.orchestrator:Starting iteration 1/2
INFO:google.adk.models.google_llm:Sending out request, model: gemini-2.0-flash, backend: ml_dev, stream: False
INFO:google.adk.models.google_llm:
LLM Request:
-----------------------------------------------------------
System Instruction:
You are an expert AI agent for AutoML. Given the dataset metadata and previous evaluation results, recommend 1-2 scikit-learn classification models and specific hyperparameter configurations to try next. If no previous results exist, suggest initial models and hyperparameters. Provide a brief reasoning for each recommendation. Respond with a JSON object: {"

KeyboardInterrupt: 